# source code

In [92]:
import os
import folium
from folium import FeatureGroup, LayerControl, Map, Marker
import time
import datetime
import re
import pandas as pd 

CSV_EXTENSION  = 'csv'
EXCEL_EXTENSION = 'xls'


class EWBLayer():
    def __init__(self, name, color, df):
        self.name = name
        self.color = color
        self.df = df
        self.points = []
        self.featureGroup = FeatureGroup(name=name)

# ref: https://github.com/python-visualization/folium/issues/460
class EWBMap():
    def __init__(self):
        self.layers = []
        self.map = Map(
            location=[45.372, -121.6972],
            zoom_start=12,
            tiles='Stamen Terrain'
        )
        listOfTiles = ['Stamen Toner', 'openstreetmap']
        for tile in listOfTiles:
            folium.TileLayer(tile).add_to(self.map) 
        self.layerNames = []
    
    def makeMapfromLayers(self):
        for layer in self.layers:
            layer.featureGroup.add_to(self.map)
        folium.LayerControl().add_to(self.map)
    
    def recenter(self, string = "all"):
        # center around the town - hardcode location TBD
        if string == 'town':
            x = 45.5236
            y = -122.6750
            self.map.location = [x, y]

        elif string in self.layerNames:
            xMin, xMax, yMin, yMax = [100, -100, 181, -181]
            idx = self.layerNames.index(string)
            layer = self.layers[idx]
            # layer.df[0] - latitutde, layer.df[1] - longitude --> I know this looks ugly
            # but is only meant to be a quick fix, will be optimized later
            xMin = min(xMin, min(layer.df.iloc[:,0]))
            xMax = max(xMax, max(layer.df.iloc[:,0]))
            yMin = min(yMin, min(layer.df.iloc[:,1]))
            yMax = max(yMax, max(layer.df.iloc[:,1]))
            # adjust the map bound according the upper and lower bound of the dataset
            self.map.fit_bounds([[xMin, yMin], [xMax, yMax]])
        
        elif string == 'all':
            xMin, xMax, yMin, yMax = [100, -100, 181, -181]
            for layer in self.layers:
                xMin = min(xMin, min(layer.df.iloc[:,0]))
                xMax = max(xMax, max(layer.df.iloc[:,0]))
                yMin = min(yMin, min(layer.df.iloc[:,1]))
                yMax = max(yMax, max(layer.df.iloc[:,1]))
            # adjust the map bound according the upper and lower bound of the dataset
            self.map.fit_bounds([[xMin, yMin], [xMax, yMax]])
        # if the input string does not match any of the options
        else:
            print(f"ERROR: wrong string input: {string}\n, valid inputs are: {self.layerNames} or 'all' or 'town")

        
def determine_extension(filename: str) -> str:
    """
    Returns type of extension constant  associated with a Filename.
    If Extension is not supperted IOError is raised.
    """
    name_pattern = r'.*'
    excel_reg = re.compile(name_pattern + r'.(xls|xlsx|xlsm|xlsb|xls)$')
    csv_reg = re.compile(name_pattern + r'.csv$')
    extension = None
        
    if excel_reg.match(filename):
        extension = EXCEL_EXTENSION
    elif csv_reg.match(filename):
        extension = CSV_EXTENSION
    else:
        error_msg = "File extension or name of " + filename
        error_msg += " not supported. Use csv or excel documents"
        raise IOError(error_msg)
    
    return extension
    
def get_dataframe(filename: str) -> pd.DataFrame:
    """
    Returns a dataframe from an excel or csv file
    """
    
    extension = determine_extension(filename)
    df = None
    if extension == EXCEL_EXTENSION:
        df = pd.read_excel(filename)
    elif extension == CSV_EXTENSION:
        df = pd.read_csv(filename)
    else:
        error_msg = "File extension or name of " + filename
        error_msg += " not supported. Use csv or excel documents"
        raise IOError(error_msg)
    # doing validation, modification, and dataframe making at the same time
    # could be a terribly bad idea but a quick fix for now --Chen
    for i in range(df.shape[0]):
        for j in range(2):
            string = df.iloc[i, j]
            if type(string) == str and '°' in string:
                df.iloc[i,j] = conv_coord(string)
    return df


def conv_coord(deg_notation):
    degrees, rest = deg_notation.split('°', 1)
    minutes, rest = rest.split("'", 1)
    seconds, direction = rest.split('"', 1)
    dec_notation = float(degrees) + float(minutes)/60 + float(seconds)/3600
    
    if direction == 'S' or direction == 'W':
        dec_notation = -dec_notation

    return dec_notation

def get_map(files: list) -> EWBMap:
    """"
    Returns a map object read from a csv or excel file
    """
    map_object = EWBMap()
    for f in files:
        layer = get_layer(f)
        map_object.layers.append(layer)
        map_object.layerNames.append(layer.name)
    map_object.makeMapfromLayers()
    map_object.recenter()
    return map_object


def get_layer(filename: str) -> EWBLayer:
    map_df = get_dataframe(filename)
    layername, color, _= re.split(',|_|-|\\.',filename.replace(' ', ''))
    layer_object = EWBLayer(layername, color, map_df)

    for row in map_df.itertuples():
        tmpRow = list(row)
        make_popups(layer_object.featureGroup, tmpRow[1], tmpRow[2], tmpRow[3], 
                    tmpRow[4], tmpRow[5], tmpRow[6], layer_object.color)

    return layer_object
   
    
def make_popups(layer, lat, lon, title, date="", description="", icon="home", color="lightgray"):
    if date == "":
        date = datetime.datetime.now()
    folium.Marker(
        location = [lat, lon],
        icon = folium.Icon(icon=icon, color=color, prefix='fa'),
        tooltip = title,
        popup = folium.Popup(
                    folium.Html('<b>%s</b> <br> <i>%s</i> <br> %s' %(title, date, description), script=True),
                    # min_width=100,=> this is no longer valid
                    max_width=450)
        ).add_to(layer)
    
    
def map_to_html (m, map_name, file_path =""):
    if file_path != "" and file_path[-1]!= '/':
        file_path = file_path+"/"
    path = file_path+map_name+".html"
    m.save(path)
    return path

# still not working :/
# we will not continue developing this function, I only keep this here for record keeping. --Chen
def map_to_png(m, map_name, file_path ="", browser= 'Chrome'):
    if file_path != "" and file_path[-1]!= '/':
        file_path = file_path+"/"
    fn = map_name+".html"
    m.save(fn)
    tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)
    delay =5
    if browser == 'Safari':
        browser = webdriver.Safari()
    elif browser == 'Firefox':
        browser = webdriver.Firefox()
    else:
        browser = webdriver.Chrome()
    options = webdriver.ChromeOptions()
    options.binary_location = "./chromedriver.exe"    #chrome binary location specified here
    options.add_argument("--start-maximized") #open Browser in maximized mode
    options.add_argument("--no-sandbox") #bypass OS security model
    options.add_argument("--disable-dev-shm-usage") #overcome limited resource problems
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    brower = webdriver.Chrome(options=options, executable_path=r'./chromedriver.exe')
    # browser.get('http://google.com/')
    
    browser.get(tmpurl)
    # time.sleep(delay)
    # browser.save_screenshot(file_path+map_name+'.png')
    # browser.quit()



# user code

In [1]:
# run this cell if notebook is executed in Google Colab
# ignore if run in jupyter notebook
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [61]:
# change the directory to where you store the notebook and the excel/csv file
%cd /content/drive/My Drive/Colab Notebooks/folium
!ls

/content/drive/My Drive/Colab Notebooks/folium
app_update.ipynb  sample_blue.xlsx  Surveyees_orange.xlsx    vertical_red.xlsx
chromedriver.exe  sampleHTML.html   Surveyees,Yellow.csv
folium.ipynb	  samplePNG.html    Surveyees,Yellow.gsheet


In [95]:
# test cases
# construction block
sampleMap = get_map(['Surveyees,Yellow.csv'])
map_to_html(sampleMap.map, 'sampleHTML', '.')
# sampleMap.map

'./sampleHTML.html'

In [96]:
# visualization block
sampleMap.recenter()
sampleMap.map